In [1]:
# Sentiment Analysis
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../enron_emails_cleaned.csv')

In [ ]:
# praeparing the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

text=df['Content'].astype(str)

In [ ]:
df['sentiment scores'] = [analyzer.polarity_scores(message) for message in text]
df.sample(5)

In [ ]:
#split based on time
df['Date'] = pd.to_datetime(df['Date'])
#choose a time
split_year=2000
split_month=6

time1_data=df[(df["Year"]<split_year)|((df['Year']==split_year) & (df["Month"]<split_month))]
time2_data=df[(df["Year"]>split_year)|((df['Year']==split_year) & (df["Month"]>=split_month))]


In [ ]:
time1_data.sample(5)


In [ ]:
time2_data.sample(5)

In [ ]:
# Determine the positive and negative scores
def determine_sentiment(data):
    sentiment = []
    for i in data['sentiment scores'].values:
        if i['compound'] > 0.05:
            sentiment.append('Positive')
        elif i['compound'] < 0.05:
            sentiment.append('Negative')
        else:
            sentiment.append('Neutral')
    data['Sentiment'] = sentiment
    return data

time1_data=determine_sentiment(time1_data)
time2_data=determine_sentiment(time2_data)

In [ ]:
sentiment_values_time1 = time1_data['Sentiment'].value_counts()
sentiment_values_time1.columns = ['Sentiment', 'Count']
sentiment_values_time2 = time2_data['Sentiment'].value_counts()
sentiment_values_time2.columns = ['Sentiment', 'Count']
#print
print("Sentiment values for time1:")
print(sentiment_values_time1)

print("\nSentiment values for time2:")
print(sentiment_values_time2)

In [ ]:
# a graph for the sentiment as a funnel shaped graph
fig_time1 = px.funnel(sentiment_values_time1, title='Sentiment Funnel for Time1')
fig_time1.show()

fig_time2 = px.funnel(sentiment_values_time2, title='Sentiment Funnel for Time2')
fig_time2.show()


#!pip install nltk==3.8.1

In [ ]:
import nltk
nltk.download('punkt')# split the sentence
nltk.download('wordnet')#textblob
nltk.download('vader_lexicon')  # VADER
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') #lexical notation    

from nrclex import NRCLex
from textblob import TextBlob

In [ ]:
#!python -m textblob.download_corpora

time1_data['Content'] = time1_data['Content'].astype(str)
time2_data['Content'] = time2_data['Content'].astype(str)

In [ ]:
# defining another function for sentiment analysis with different emotions using NRCLex lexicon
def sentiment(message):
    text = NRCLex(message)
    if text.top_emotions[0][1] == 0.0:
        return 'No Emotion'
    else:
        return text.top_emotions[0][0]
time1_data['Emotions'] =  time1_data['Content'].apply(sentiment)
time2_data['Emotions'] =  time2_data['Content'].apply(sentiment)

time1_data.sample(5)
time2_data.sample(5)

In [ ]:
emotions_time1 = time1_data['Emotions'].value_counts()
emotions_time2 = time2_data['Emotions'].value_counts()

emotion_chart_time1 = pd.DataFrame(emotions_time1)
emotion_chart_time2 = pd.DataFrame(emotions_time2)

emotion_chart_time1 = emotion_chart_time1.drop('No Emotion', axis = 0)
emotion_chart_time2 = emotion_chart_time2.drop('No Emotion', axis = 0)

emotion_chart_time1, emotion_chart_time2

In [ ]:
# A pie chart for showing the percentage of every emotion in the text
labels_time1 = emotion_chart_time1.index.tolist()
labels_time2 = emotion_chart_time2.index.tolist()

fig, axs=plt.subplots(1,2,figsize=(20,10))

axs[0].pie(emotion_chart_time1['count'].values, labels = labels_time1 , autopct = '%1.1f%%',labeldistance= 1.1)
axs[0].set_title('Emotions - Time1', fontsize=14)

axs[1].pie(emotion_chart_time2['count'].values, labels = labels_time2 , autopct = '%1.1f%%',labeldistance= 1.1)
axs[1].set_title('Emotions - Time2', fontsize=14)

plt.legend(loc = 'upper right')
plt.show()